In [1]:
import music21

In [8]:
import random




In [4]:
import os

In [ ]:
os.

In [3]:
import pickle as pkl

In [5]:
%%writefile midi_preproc.py

from music21 import converter, stream, note, chord, duration, pitch
from scipy.sparse import csc_matrix

import os.path


import pickle as pkl

import numpy as np

instruments = ['Piano', 'Guitar', 'Violin']

def midiToMatrix(filename):
    if os.path.isfile(filename+'.prep') :
        f = open(filename+'.prep', 'rb')
        return pkl.load(f)
    parsed = converter.parse(filename)

    party = []

    for part in parsed:
        for voice in part.getElementsByClass(stream.Voice):
            if voice.getInstrument().instrumentName in instruments:
                for thisNote in [n for n in voice if (isinstance(n,note.Note) or isinstance(n,chord.Chord))]:
                    cur_chord = np.zeros(129)
                    for _pitch in thisNote.pitches:
                        cur_chord[_pitch.midi] = 1
                        #text += pitch.name+str(pitch.octave)
                    dur = thisNote.duration.quarterLength
                    cur_chord[-1] = dur#dur.numerator / float(dur.denominator)

                    party.append(cur_chord)
                    #text += dur_to_text(thisNote.duration.type)+'z'
    res= csc_matrix(party) if len(party) > 0 else csc_matrix(np.zeros((0,129)))
    with open(filename+'.prep', 'wb') as f:
        pkl.dump(res,f)
    return res;

from music21 import midi

def save_mat2_mid(mat, fname='output/test.mid'):
    music_stream = stream.Stream()

    for dense_line in np.array(mat):
        (notes,) = np.where(dense_line[:-1]>0.5)

        pitches = []
        for n in notes:
            pitches.append(pitch.Pitch(midi=n))

        crd = chord.Chord(notes= pitches, quarterLength=np.round(dense_line[-1]*2048)/2048)
        music_stream.append(crd)
        
    md = midi.translate.streamToMidiFile(music_stream)
    md.open(fname, 'wb')
    md.write()
    md.close()

Overwriting midi_preproc.py


In [6]:
#test
sparsed = midiToMatrix('midi/abide_.mid')

In [7]:
save_mat2_mid(sparsed)

In [4]:
%%writefile dataset_generation.py

SEQ_LEN = 200
STRIDE_MU = 20 #expected value of stride
STRIDE_SIG = 2 #standard standard deviation of stride

import sys
import os
from os import listdir
from random import choice
from midi_preproc import midiToMatrix
import numpy as np
import scipy
from prefetch_generator import background
import pickle as pkl

import tqdm as tqdm
def parallelization(fun,massive,tq = True):    
    num_cores = 20#multiprocessing.cpu_count()
    if tq:
        results = np.array(Parallel(n_jobs=num_cores)(delayed(fun)(i) for i in tqdm(massive)))
        return results
    else:
        results = Parallel(n_jobs=num_cores)(delayed(fun)(i) for i in massive)
        return results

bad_files = []

if os.path.isfile('bad_files.pkl'):
    with open('bad_files.pkl','rb') as f:
        bad_files = pkl.load(f)

#all files in storage
files = [each for each in listdir('midi') if each.endswith('.mid') and (not ('midi/'+each in bad_files))]
files_precomp = [each for each in files if os.path.isfile("midi/"+each+'.prep')]


def split_random_file(precomputed=False):#returns sparse matrix every SEQ_LEN rows of wich are cut from the vectorized midi file
    fi = choice(files_precomp) if precomputed else choice(files)#select random midi file
    
    cut = []
    
    sparsed = midiToMatrix('midi/'+fi)
    pos = 0
    
    #cut the sequence
    while(sparsed.shape[0] - pos > SEQ_LEN):
        
        cut.append(sparsed[pos: pos+SEQ_LEN])
        
        pos+= np.clip(np.round(np.random.normal(STRIDE_MU, STRIDE_SIG)), 1, SEQ_LEN//2)
    if len(cut) > 0:
        return scipy.sparse.vstack(cut,format='csr')
    else:
        return scipy.sparse.csc_matrix(np.zeros(shape=(0,129)))
    
def generate_minibatch(batch_size=32, precomputed=True):
    res = []
    while(sum([x.shape[0] for x in res])/SEQ_LEN <= batch_size):
        res.append(split_random_file(precomputed))
    return scipy.sparse.vstack(res,format='csr')

#@background(max_prefetch=10)
def iterate_minibatches(num_batches, batch_size=32, precomputed=True):
    for i in xrange(num_batches):
        yield generate_minibatch(batch_size,precomputed)

Overwriting dataset_generation.py


In [1]:
from dataset_generation import generate_minibatch

In [31]:
test_rnd_cutter = split_random_file()

In [3]:
test_generate = generate_minibatch()

ress


In [4]:
test_generate

<8200x129 sparse matrix of type '<type 'numpy.float64'>'
	with 25189 stored elements in Compressed Sparse Row format>

In [6]:
%%time
midi_preproc.midiToMatrix('midi/abide_.mid')

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 133 ms


<2110x129 sparse matrix of type '<type 'numpy.float64'>'
	with 5031 stored elements in Compressed Sparse Column format>

In [2]:
from os import listdir

In [5]:
import glob
import os
import pickle as pkl

bad_files= []

if os.path.isfile('bad_files.pkl'):
    with open('bad_files.pkl','rb') as f:
        bad_files = pkl.load(f)

files = glob.glob('midi/*/*/*/*.mid')#[each for each in listdir('midi') if each.endswith('.mid') and (not ('midi/'+each in bad_files))]
files = [each for each in files if not os.path.isfile(each+'.prep') and (not each in bad_files)]


In [6]:
len(files)

109275

In [7]:
from tqdm import tqdm_notebook
from multiprocessing import Pool
import midi_preproc
import signal

def handler(signum, frame):
    raise Exception("Execution timed out")

signal.signal(signal.SIGALRM, handler)
    
def preprocess(fname):
    try:
        signal.alarm(3.5*60)
        res = midi_preproc.midiToMatrix(fname)
        signal.alarm(0)
        if res.shape[0] == 0:
            return False, fname
        del res
        return True, fname
    except BaseException:
        signal.alarm(0)
        return False, fname

all_goods = 0



for fname in tqdm_notebook(files):
    good, _ = preprocess(fname)
    if not good:
        bad_files.append(fname)

In [ ]:


from tqdm import tqdm_notebook
from multiprocessing import Pool
import midi_preproc
import signal

def handler(signum, frame):
    raise Exception("Execution timed out")

signal.signal(signal.SIGALRM, handler)
    
def preprocess(fname):
    try:
        signal.alarm(4*60)
        res = midi_preproc.midiToMatrix(fname)
        signal.alarm(0)
        if res.shape[0] == 0:
            return False, fname
        del res
        return True, fname
    except BaseException:
        signal.alarm(0)
        return False, fname


all_goods = 0
if __name__ == '__main__':
    p = Pool(3,maxtasksperchild=1)
    for good, fname in tqdm_notebook(p.imap_unordered(preprocess, files),total=len(files)):
        if not good:
            bad_files.append(fname)
            if len(bad_files) % 10 == 9:
                with open("bad_files.pkl",'wb') as f:
                    pkl.dump(bad_files, f)
        else:
            all_goods+=1

In [8]:
import pickle as pkl

In [9]:
with open("bad_files.pkl",'wb') as f:
    pkl.dump(bad_files, f)

In [11]:
len(bad_files)

85